# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [3]:
def euclidean_distance(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension):
        dist += math.pow(a[i]-b[i],2)
    return math.sqrt(dist)
    
    
    
def euclidean_distance_assymetrical(a,b)->float:  
    """ 
    points as a 
    tuple or list for example a = (1,2,3) , b = (4,5,6)
    """
    import math
    dimension = len(a)
    if (dimension !=len(b)):
        print("Dimension missmatch")
        return 0 
    dist = 0
    for i in range(dimension-1): # let's assume last dimesion as high 
        dist += math.pow(a[i]-b[i],2)
        
    if ((a[-1]-b[-1]) > 0):   # a is higher than b so distance from a to b 
        dist += math.pow(0.9*(a[-1]-b[-1]),2)
    else:
        dist += math.pow(1.1*(a[-1]-b[-1]),2)
    
    return math.sqrt(dist)
    

In [4]:
def name_city(n):
    """Name cities in Excel convention starting from A after Z going to AA then AAA and so on 
            Then calculating distances.
    """
    name = ""
    while True:
        
        rest = n % 26  # getting so called rest and then convert to letter
        name = chr(65+rest) + name # add to already mapped values
        
        
        n  = n // 26 - 1 # move on to 'next' char -1 is neccessary because we have some problem with index without it 26 is mapped to 0->A at last position but 26 // 26 -> 1 at first 

        if (n < 0 ):        # will be fullfiled when we iterate through last chat -> n // 26 - 1  = 0 -1 = -1 
            break
    return name
        
    


In [5]:
euclidean_distance((0,0,0),(0,0,4))

4.0

In [6]:
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        import random
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        
        self.cities = []
        self.routes = dict()
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(-51,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(-51,51)))
            len_check = len(set(self.cities))
        self.cities = set(self.cities)
        

        
    def create_routes_symmetrical(self, connections = 1):
        """Creates basic graph symmetrical . Default all connections, however setting connections to 0.5 will result in approx 1/2 connections possible connections 
        """
        # for now example with connections to every city
        cities = list(self.cities)
        indexes = len(cities)
                   
                
        if (connections==1):   
            for i in range(indexes):
                for j in range(i+1 , indexes):  # to not double calcualte
                    name_i = name_city(i)
                    name_j = name_city(j)
                    distance = euclidean_distance(cities[i], cities[j])
                
                    # Store the route only once, as it is undirected
                    self.routes[(name_i, name_j)] = distance
            return self.routes
        else:
            import random
            # calculate number of citeis from which to write connections 
            cons =  indexes * connections
            # random sampling 
            index_cons = random.sample(range(0,indexes),int(cons))
            
            # same as previous but now first loop only iterate through previously generated indexes
            # however in this method we'll be getting quite some variatons because sampling low indexes will result in getting more connections then sampling from high indexes
            
            for i in index_cons:
                for j in range(i+1 , indexes):  # to not double calcualte
                    name_i = name_city(i)
                    name_j = name_city(j)
                    distance = euclidean_distance(cities[i], cities[j])
                
                    # Store the route only once, as it is undirected
                    self.routes[(name_i, name_j)] = distance
            return self.routes
            
            
    def create_routes_asymmetrical(self, connections = 1):
        """Creates assymetrical graph.
            Now he have to consider that going from city located 'lower' to 'higher' and vice versa is not the same
        Default all connections, however setting connections to 0.5 will result in approx 1/2 connections possible connections
        """
        
        # for now example with connections to every city
        cities = list(self.cities)
        indexes = len(cities)
                   
                
        if (connections==1):   
            for i in range(indexes):
                for j in range(indexes):  # now we neeed to calculate routes from a to b aswell as from b to a 
                    name_i = name_city(i)
                    name_j = name_city(j)
                    distance = euclidean_distance_assymetrical(cities[i], cities[j])
                
                    if distance !=0:
                        self.routes[(name_i, name_j)] = distance
            return self.routes
        else:
            import random
            import itertools

            # calculate number of citeis from which to write connections 
            all_cons = list(itertools.combinations(cities,2))  # all cities
            
            cons =int(indexes * connections) 
            # random sampling 
            selected_connetions = random.sample(all_cons,cons)
            length = len(selected_connetions)
            
            # same as previous but now first loop only iterate through previously generated indexes
            # however in this method we'll be getting quite some variatons because sampling low indexes will result in getting more connections then sampling from high indexes
            
            
            for a,b in selected_connetions:
                cost_ab = euclidean_distance_assymetrical(a, b)
                cost_ba = euclidean_distance_assymetrical(a, b)
                city_a = name_city(a)
                city_b = name_city(b)
                self.routes[(city_a,city_b)] = cost_ab
                self.routes[(city_a,city_b)] = cost_ba  # For asymmetric travel costs
        
            return self.routes
    
    def print_cities(self):
        print(list(self.cities))
        
        

In [7]:
test = CitiesMap(20,seed=42)
test.create_routes_symmetrical(connections=0.8)

{('C', 'D'): 147.50254235097103,
 ('C', 'E'): 136.95254652615992,
 ('C', 'F'): 147.35331689514152,
 ('C', 'G'): 134.06341782902598,
 ('C', 'H'): 50.80354318352215,
 ('C', 'I'): 135.26640381114595,
 ('C', 'J'): 163.5145253486674,
 ('C', 'K'): 130.50670480860362,
 ('C', 'L'): 150.69173832695674,
 ('C', 'M'): 140.31393373432306,
 ('C', 'N'): 114.03508232118746,
 ('C', 'O'): 131.21737689803132,
 ('C', 'P'): 138.74797295816614,
 ('C', 'Q'): 156.8980560746372,
 ('C', 'R'): 77.1621668954417,
 ('C', 'S'): 106.9579356569675,
 ('C', 'T'): 79.68688725254614,
 ('R', 'S'): 82.86132994346639,
 ('R', 'T'): 113.94735626595292,
 ('J', 'K'): 157.9271984174987,
 ('J', 'L'): 107.57787876696584,
 ('J', 'M'): 83.83913167489273,
 ('J', 'N'): 78.8986691902975,
 ('J', 'O'): 112.16505694733989,
 ('J', 'P'): 118.75184209097559,
 ('J', 'Q'): 155.92947123619703,
 ('J', 'R'): 98.12746812182611,
 ('J', 'S'): 164.56305782282973,
 ('J', 'T'): 179.6914021315433,
 ('L', 'M'): 51.65268628057983,
 ('L', 'N'): 50.039984012

In [8]:
test2 = CitiesMap(20,seed=42)

test2.create_routes_asymmetrical(connections=1)

{('A', 'B'): 143.65430727966356,
 ('A', 'C'): 85.6789939249989,
 ('A', 'D'): 75.78845558526707,
 ('A', 'E'): 102.51833982268735,
 ('A', 'F'): 131.9189523912315,
 ('A', 'G'): 191.35422650153302,
 ('A', 'H'): 109.77563481938968,
 ('A', 'I'): 79.16697543799434,
 ('A', 'J'): 154.7962854851498,
 ('A', 'K'): 45.043201484796796,
 ('A', 'L'): 96.5828659752857,
 ('A', 'M'): 86.4190372545309,
 ('A', 'N'): 77.49380620410899,
 ('A', 'O'): 67.50437022889703,
 ('A', 'P'): 190.97528635925644,
 ('A', 'Q'): 111.72023093424038,
 ('A', 'R'): 59.179810746571334,
 ('A', 'S'): 51.815152224035785,
 ('A', 'T'): 132.96920696161197,
 ('B', 'A'): 130.77522701184657,
 ('B', 'C'): 161.40820920882555,
 ('B', 'D'): 100.18787351770672,
 ('B', 'E'): 87.34008243641632,
 ('B', 'F'): 70.55671477612886,
 ('B', 'G'): 161.1420491367787,
 ('B', 'H'): 134.67000408405727,
 ('B', 'I'): 80.63553807100192,
 ('B', 'J'): 27.10885464197999,
 ('B', 'K'): 134.1365349187163,
 ('B', 'L'): 84.76679774534367,
 ('B', 'M'): 60.5459329765426

In [9]:
test3 = CitiesMap(5,seed=42)

test3.create_routes_asymmetrical(connections=0.8)

TypeError: unsupported operand type(s) for %: 'tuple' and 'int'